In [ ]:
import ipyvuetify as v
import ipywidgets as widgets
from utils import utils
from scripts import run_gee_process

In [ ]:
#for testing 
#year = '2019'
#asset_name = "users/bornToBeAlive/Glad_Singapore"

In [ ]:
#messages
NO_INPUT = "the input are not correctly set up, go back to step 1"

In [ ]:
gee_process_output = v.Alert(children=[''], type='info', text=True, class_="mt-5")

In [ ]:
#validate the selected data
gee_process_btn = v.Btn(
    color='primary', 
    children=[
        v.Icon(left=True, children=['mdi-cogs']),
        'Run GEE process'
    ]
)
gee_validate_data = v.Container(children=[gee_process_btn])

def gee_on_click(widget, event, data):
    
    global gee_process_output
    global asset_name
    global year
    global gee_process_btn
    
    if asset_name == None or year == None:
        utils.displayIO(gee_process_output, 'error', NO_INPUT)
        return 
    
    #do something 
    utils.toggleLoading(gee_process_btn)
    run_gee_process.run_GEE_process(asset_name, year, gee_process_output)
    utils.toggleLoading(gee_process_btn)

gee_validate_data.on_event('click', gee_on_click)

In [ ]:
#create the display 
inputs = v.Layout(
    _metadata={'mount-id': 'data-input'},
    row=True,
    class_="pa-5",
    align_center=True, 
    children=[
        v.Flex(xs12=True, children=[gee_validate_data]),
        v.Flex(xs12=True, children=[gee_process_output])
    ]
)
#inputs

GEE_content_main =  v.Layout(
    _metadata={'mount_id': 'gee_process_widget'},
    row=True,
    xs12=True,
    align_center=True, 
    class_="ma-5",
    children=[
        v.Card( 
            class_="pa-5",
            raised=True,
            xs12=True,
            children=[
                v.Html(xs12=True, tag='h2', children=['Run the GEE process']),
                v.Flex(xs12=True, lg6=True, children=[inputs]),   
            ]
        )
    ]
)
GEE_content_main